# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# print document in order to index appropriately
pprint(establishments.find_one())

In [ ]:
# Find the establishments with a hygiene score of 20
hygiene_query = {"scores.Hygiene" : {"$eq" : 20}}

# Use count_documents to display the number of documents in the result
query_count = establishments.count_documents(hygiene_query) 
print(f"There are {query_count} documents that have a hygiene score of 20.")

# Display the first document in the results using pprint
pprint(establishments.find_one(hygiene_query))

In [ ]:
# Convert the result to a Pandas DataFrame
ratings = establishments.find({"scores.Hygiene" : {"$eq" : 20}})
ratings_df = pd.DataFrame(list(ratings))
ratings_df.set_index("_id", inplace=True)

# Display the number of rows in the DataFrame
print(f"There are {len(ratings_df)} rows within the dataframe.")

# Display the first 10 rows of the DataFrame
ratings_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
london_query = {"LocalAuthorityName" : {"$regex" : "London"}, "RatingValue" : {"$gte" : 4}}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(london_query)} documents that have London as the Local Authority and also a rating of 4 or more.")

# Display the first document in the results using pprint
pprint(establishments.find_one(london_query))

In [ ]:
# Convert the result to a Pandas DataFrame
london = establishments.find(london_query)
london_df = pd.DataFrame(list(london))
london_df.set_index("_id", inplace=True)

# Display the number of rows in the DataFrame
print(f"There are {len(london_df)} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
london_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
penang = establishments.find_one({"BusinessName" : "Penang Flavours"})
latitude = penang["geocode"]["latitude"]
longitude = penang["geocode"]["longitude"]

degree_search = 0.01

query = {"geocode.latitude" : {"$gte" : latitude - degree_search, "$lte" : latitude + degree_search, "$ne" : latitude},
    "geocode.longitude" : {"$gte" : longitude - degree_search, "$lte" : longitude + degree_search, "$ne" : longitude},
    "RatingValue": {"$eq" : 5}}


# Rating value must equal 5
rating = {"RatingValue": {"$eq" : 5}}

# Sort by hygiene score
sort = [("scores.Hygiene", 1)]

limit = 5

results = establishments.find(query).sort(sort).limit(limit)

# Print the results
for r in range(5):
    pprint(results[r])


In [ ]:
# Convert result to Pandas DataFrame
top_closest_df = pd.DataFrame(list(results))
top_closest_df.set_index("_id", inplace=True)

In [ ]:
top_closest_df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# 1. Matches establishments with a hygiene score of 0
match_query = {"$match": {"scores.Hygiene" : 0}}

# 2. Groups the matches by Local Authority
group_query = {"$group" : {"_id" : "$LocalAuthorityName", "count" : {"$sum" : 1}}}

# 3. Sorts the matches from highest to lowest
sort = {"$sort" : {"count" : -1}}

# Create a pipeline
pipeline = [match_query, group_query, sort]

# Print the number of documents in the result
count = list(establishments.aggregate(pipeline))
print(f"There are {len((count))} local authorities within this subset that contain establishments with the best hygiene.")

# Print the first 10 results
pprint((count[:10]))

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(count)
hygiene_df.rename(columns={"_id" : "Local Authority", "count" : "Number of Establishments"}, inplace=True)
hygiene_df.set_index("Local Authority", inplace=True)

# Display the number of rows in the DataFrame
print(f"There are {len(hygiene_df)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)